In [ ]:
# http://eddmann.com/posts/depth-first-search-and-breadth-first-search-in-python/

In [ ]:
#dfs stack那个方法就是由两条线组成，一个是visited那个那个set组成的路径， 
                              #另一个是stack组成的，已访问node所能链接到的node（不包括visited里出现的node)
# stack.pop()那里可能连续弹出两个，如果所到node不延伸出新的node。这种情况下，后面的if条件判断不成立，于是有弹出一个

In [ ]:
append不需要重新赋值给原来的list
mylist=mylist.append('X')是错的。
下面两个操作是不equivalent的
exploredNodes.append("x")
exploredNodes = exploredNodes + ["x"]

In [23]:
#Bug, the default value is evaluated only once
# soruce:https://developmentality.wordpress.com/2010/08/23/python-gotcha-default-arguments/
def f(a, L=[]):
  L.append(a)
  return L
 
print f(1)
print f(2)



[1]
[1, 2]


In [26]:
def f(a, L=[]):
  L=L+[a]  # assignment这个操作使得，L这个variable的scope只在这个function里。
  return L
 
print f(1)
print f(2)

[1]
[2]


In [ ]:
#这个bug在search all paths里的体现： 这个function,连续调用几次，结果不同，因为default argument path被刷新了
def find_all_paths(graph, start, end, path=[]):
  path.append([start])
  if start == end:
    return [path]
  if not graph.has_key(start):
    return []
  paths = []
  for node in graph[start]:
    if node not in path:
      # recursive call
      newpaths = find_all_paths(graph, node, end, path)
      for newpath in newpaths:
        paths.append(newpath)
  return paths

In [ ]:
#找到所有可以历遍的node, 一个是用stack一个是用recursion (dfs)

In [18]:
graph1 = {'A': set(['B', 'C']),
         'B': set(['A', 'D', 'E']),
         'C': set(['A', 'F']),
         'D': set(['B']),
         'E': set(['B', 'F']),
         'F': set(['C', 'E'])}

In [4]:
#stack 方法
def dfs(graph, start):
    visited, stack = set(), [start]
    while stack:
        vertex = stack.pop()
        if vertex not in visited:
            visited.add(vertex)
            print vertex
            stack.extend(graph[vertex] - visited)
    return visited

dfs(graph1, 'A') # {'E', 'D', 'F', 'A', 'C', 'B'}

A
B
D
E
F
C


{'A', 'B', 'C', 'D', 'E', 'F'}

In [18]:
stack=[]
print graph['A']
stack.extend(graph['A'])
print stack
stack.extend(graph['B'])
print stack

set(['C', 'B'])
['C', 'B']
['C', 'B', 'A', 'E', 'D']


In [15]:
# recursion
def dfs(graph, start, visited=None):
    if visited is None:
        visited = set()
    visited.add(start)
    for next in graph[start] - visited:
        print graph[start],next
        dfs(graph, next, visited)
    return visited

dfs(graph, 'A') 

set(['C', 'B']) C
set(['A', 'F']) F
set(['C', 'E']) E
set(['B', 'F']) B
set(['A', 'E', 'D']) D
set(['C', 'B']) B


{'A', 'B', 'C', 'D', 'E', 'F'}

In [ ]:
dfs(g,"A",None)
dfs(g,"C",[A]) A的链接的node有C和B  #这一层for loop不一定到最后执行完，这里B node被后来从E链接到了，
                                 #而下面一个graph，由于C不链接到其它node，B和C 都别历遍到了。
dfs(g,"F",[A,C]) C链接的node有A和F 

dfs(g,"E",[A,C,F]) F的链接有E和C


In [16]:
graph2 = {'A': set(['B', 'C']),
         'B': set(['A', 'D', 'E']),
         'C': set(['A']),
         'D': set(['B']),
         'E': set(['B'])}

In [17]:
dfs(graph2,'A')

set(['C', 'B']) C
set(['C', 'B']) B
set(['A', 'E', 'D']) E
set(['A', 'E', 'D']) D


{'A', 'B', 'C', 'D', 'E'}

In [12]:
# return 所有的pathes

In [22]:
def dfs_paths(graph, start, goal):
    stack = [(start, [start])]
    i=1
    while stack:
        print stack
        (vertex, path) = stack.pop()
        for next in graph[vertex] - set(path):
            if next == goal:
                yield path + [next]
                print "find",i,"th path"
                i+=1
            else:
                stack.append((next, path + [next]))

list(dfs_paths(graph1, 'A', 'F')) # [['A', 'C', 'F'], ['A', 'B', 'E', 'F']]

[('A', ['A'])]
[('C', ['A', 'C']), ('B', ['A', 'B'])]
[('C', ['A', 'C']), ('E', ['A', 'B', 'E']), ('D', ['A', 'B', 'D'])]
[('C', ['A', 'C']), ('E', ['A', 'B', 'E'])]
find 1 th path
[('C', ['A', 'C'])]
find 2 th path


[['A', 'B', 'E', 'F'], ['A', 'C', 'F']]

In [ ]:
[('A', ['A'])] 
[('C', ['A', 'C']), ('B', ['A', 'B'])] 
[('C', ['A', 'C']), ('E', ['A', 'B', 'E']), ('D', ['A', 'B', 'D'])] #（'B',['A','B'])被弹出，
                                                              # E和D两个node加上
[('C', ['A', 'C']), ('E', ['A', 'B', 'E'])] # D 被弹出，但不增加其它node.
[('C', ['A', 'C'])] #E 被弹出，找到F,

In [25]:
# Look up cycle:
#source: https://algocoding.wordpress.com/2015/04/02/detecting-cycles-in-an-undirected-graph-with-dfs-python/
graph3 = { 0 : [1],
                  1 : [0, 2, 3, 5],    # edit: corrected from [0, 2] to [0, 2, 3, 5]
                  2 : [1],
                  3 : [1, 4],
                  4 : [3, 5],
                  5 : [1, 4] }

In [26]:
def cycle_exists(G):        # - G is an undirected graph.              
    marked = { u : False for u in G }  # - All nodes are initially unmarked.
    found_cycle = [False]    # - Define found_cycle as a list so we can change
                            # its value per reference, see:
    # http://stackoverflow.com/questions/11222440/python-variable-reference-assignment
 
    for u in G:              # - Visit all nodes.
        if not marked[u]:
            dfs_visit(G, u, found_cycle, u, marked)
               # - u is its own predecessor initially
        if found_cycle[0]:
            break
    return found_cycle[0]
 
 
def dfs_visit(G, u, found_cycle, pred_node, marked):
    if found_cycle[0]:                                # - Stop dfs if cycle is found.
        return
    marked[u] = True                                  # - Mark node.
    for v in G[u]:      # - Check neighbors, where G[u] is the adjacency list of u.
        if marked[v] and v != pred_node:# - If neighbor is marked and not predecessor,
            found_cycle[0] = True                     # then a cycle exists.
            return
        if not marked[v]:                             # - Call dfs_visit recursively.
            dfs_visit(G, v, found_cycle, u, marked)

In [27]:
cycle_exists(graph3)

True

In [ ]:
# topological sort
# source: http://stackoverflow.com/questions/15038876/topological-sort-python

In [ ]:
#这里递归的核心要义：
children = graph.get(node, None)  # 如过没有children，返回一个None
        level = 0 if not children else \  # if not children的情况是这个递归的起点
        (1 + max(level_evaluation(lnode) for lnode in children)) 
        # 如果有children,那个该node的level是它的所有children的level中的最大值加1.

In [1]:
graph_unsorted = [(2, []),
                  (5, [11]),
                  (11, [2, 9, 10]),
                  (7, [11, 8]),
                  (9, []),
                  (10, []),
                  (8, [9]),
                  (3, [10, 8])]

In [2]:
graph_unsorted = dict(graph_unsorted)

In [3]:
graph_sorted = []

In [4]:
from collections import defaultdict
from itertools import takewhile, count

def sort_topologically(graph):
    levels_by_node = {}
    nodes_by_level = defaultdict(set)

    def level_evaluation(node):
        if node in levels_by_node:
            return levels_by_node[node]
        children = graph.get(node, None)
        level = 0 if not children else \
        (1 + max(level_evaluation(lnode) for lnode in children))
        levels_by_node[node] = level
        nodes_by_level[level].add(node)
        return level

    for node in graph:
        level_evaluation(node)

    return list(takewhile(lambda x: x is not None,\
                          (nodes_by_level.get(i, None) for i in count()))),nodes_by_level


graph = {
        1: [2, 3],
        2: [4, 5, 6],
        3: [4,6],
        4: [5,6],
        5: [6],
        6: []
    }

result,nodes_by_level= sort_topologically(graph)

In [5]:
nodes_by_level

defaultdict(set, {0: {6}, 1: {5}, 2: {4}, 3: {2, 3}, 4: {1}})

In [16]:
print graph.get(1,None)


[2, 3]


In [19]:
from collections import defaultdict
from itertools import takewhile, count

def sort_topologically_stackless(graph):
    levels_by_name = {}
    names_by_level = defaultdict(set)

    def add_level_to_name(name, level):
        levels_by_name[name] = level
        names_by_level[level].add(name)


    def walk_depth_first(name):
        stack = [name]
        while(stack):
            name = stack.pop()
            if name in levels_by_name:
                continue

            if name not in graph or not graph[name]:
                level = 0
                add_level_to_name(name, level)
                continue

            children = graph[name]

            children_not_calculated = [child for child in children if child not in levels_by_name]
            if children_not_calculated:
                stack.append(name)
                stack.extend(children_not_calculated)
                continue

            level = 1 + max(levels_by_name[lname] for lname in children)
            add_level_to_name(name, level)

    for name in graph:
        walk_depth_first(name)

    return list(takewhile(lambda x: x is not None, (names_by_level.get(i, None) for i in count())))


graph = {
        1: [2, 3],
        2: [4, 5, 6],
        3: [4,6],
        4: [5,6],
        5: [6],
        6: []
    }

print(sort_topologically_stackless(graph))

[set([6]), set([5]), set([4]), set([2, 3]), set([1])]
